# 🗺️ Planners (with Handlebars planner)

### Initialize the kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.15.0"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.15.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.15.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.15.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.15.0-preview"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, 1.15.0-preview"

#!import ../../Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var kernel = Kernel.CreateBuilder()
    // .AddAzureOpenAIChatCompletion(
    //     deploymentName: Secrets.DeploymentName,
    //     endpoint: Secrets.Endpoint,
    //     apiKey: Secrets.ApyKey)
    .AddOpenAIChatCompletion(
        modelId: Secrets.OpenAIModel,
        apiKey: Secrets.OpenAIApiKey)    
    .Build();

### 🔌 Let's add a native plugin for use by the planner

In [ ]:
#!import ../../Helpers/PrintHelper.cs
#!import ../../Plugins/MotorPlugin.cs

kernel.Plugins.AddFromType<MotorPlugin>();

PrintHelper.PrintAllPluginFunctions(kernel);

### 📋 Prepare kernel arguments

In [ ]:
var goal = "You have a tree in front of the car. Avoid it.";
var kernelArguments = new KernelArguments()
{
    ["input"] = goal,
    ["commands"] = "go forward, go backward, turn left, turn right, and stop"
};

### Refine the goal

In [ ]:
var commandsPlugin = kernel.CreatePluginFromPromptDirectory(System.IO.Path.Combine(System.IO.Directory.GetCurrentDirectory(), "../../Plugins", "CommandsPlugin"), "CommandsPlugin");
kernel.Plugins.Add(commandsPlugin);
PrintHelper.PrintAllPluginFunctions(kernel);

In [ ]:
var semanticFunction = commandsPlugin["ExtractBasicCommands"];
var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);
var refinedGoal = response.GetValue<string>();

kernelArguments["input"] = refinedGoal;

display(refinedGoal)

### 🗺️ Generate Handlebars Plan

In [ ]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var handlebarsPlannerOptions = new HandlebarsPlannerOptions 
{ 
    AllowLoops = true 
};
handlebarsPlannerOptions.ExcludedPlugins.Add("CommandsPlugin");
var planner = new HandlebarsPlanner(handlebarsPlannerOptions);
var plan = await planner.CreatePlanAsync(kernel, refinedGoal);

//Console.WriteLine($"The proposed plan in Handlebars format:\n{plan}");

### 🗺️🚲 Let's run the plan!

(Note that the final output is json but the planner strips the extra JSON syntax)

In [ ]:
#pragma warning disable SKEXP0060

var planResult = await plan.InvokeAsync(kernel, kernelArguments);
Console.WriteLine(planResult.Trim());